## load libraries

In [1]:
import pandas as pd
import geopandas as gpd
import folium

from shapely.geometry import Polygon, Point
import numpy as np
import xarray as xr
import random
import os
import time
import re
import math

import matplotlib.pyplot as plt
import altair as alt

In [2]:
# Set GeoPandas to use pyogrio
gpd.options.io_engine = "pyogrio"

## load crop sequence boundaries

In [3]:
sample_data_file = '../data/fields/geo_balanced_sample.parquet'
geo_balanced_csb_samples = gpd.read_parquet(sample_data_file)
geo_balanced_csb_samples

,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,INSIDE_X,INSIDE_Y,Shp_Len,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop
478801,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,-6.035779e+05,1.704774e+06,532.211689,15343.877410,"MULTIPOLYGON (((-603525.799 1704829.066, -6035...",-102.962080,38.176922,1231,#bfbf7a,Fallow/Idle Cropland
107446,351623001627247,1623,4.325446,225,1,225,152,152,152,36,...,-7.967410e+05,1.149389e+06,723.964572,17504.529797,"MULTIPOLYGON (((-796679.747 1149498.895, -7966...",-104.600667,33.089835,1138,#ffa8e3,Alfalfa
121162,351623002777545,1623,3.222010,61,176,1,176,36,36,1,...,-1.086137e+06,1.590736e+06,549.623304,13039.063523,"MULTIPOLYGON (((-1086091.187 1590803.212, -108...",-108.309751,36.728108,1676,#ffd400,Corn
88813,351623000046581,1623,10.204036,36,36,36,36,152,37,37,...,-9.788489e+05,1.595846e+06,1274.741723,41294.433943,"MULTIPOLYGON (((-978851.432 1595726.11, -97884...",-107.112180,36.891083,2252,#a5f58d,Other Hay/Non Alfalfa
516266,081623013377366,1623,2.918481,24,29,29,24,1,61,61,...,-6.203751e+05,1.992249e+06,556.566951,11810.721459,"MULTIPOLYGON (((-620252.778 1992330.141, -6202...",-103.412893,40.723858,1330,#bfbf7a,Fallow/Idle Cropland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119071,351623002568175,1623,2.957066,176,176,176,61,24,24,24,...,-9.138551e+05,1.348095e+06,599.379620,11966.871324,"MULTIPOLYGON (((-913829.788 1348147.68, -91380...",-106.082615,34.756107,1877,#e9ffbe,Grass/Pasture
540278,081623014401456,1623,9.374885,61,24,61,1,1,1,1,...,-6.892023e+05,1.934718e+06,988.353218,37938.965442,"MULTIPOLYGON (((-689103.22 1934811.717, -68911...",-104.172849,40.161427,1417,#a5f58d,Other Hay/Non Alfalfa
591949,041623013556134,1623,3.770585,72,72,72,190,190,190,190,...,-1.707418e+06,1.273220e+06,523.387689,15259.076277,"MULTIPOLYGON (((-1707373.732 1273277.961, -170...",-114.508535,33.028054,101,#80b3b3,Woody Wetlands
217842,081623005639708,1623,4.307316,61,24,1,61,24,29,61,...,-6.004245e+05,1.898598e+06,634.862780,17431.160158,"MULTIPOLYGON (((-600373.153 1898692.428, -6003...",-103.092389,39.903658,1396,#a87000,Winter Wheat


## load ecocrop data

Imputed, includes USDA Hardiness Zones

In [4]:
ecocrop_file = '../data/crops/EcoCrop_Complete.pickle'
ecocrop_all = pd.read_pickle(ecocrop_file)
ecocrop_all

,Crop_Code,Scientific_Name,Genus,Species,Variety,Life_Form,Habit,Life_Span,Physiology,Category,...,Crop_Cycle_Max,Use_Main,Use_Detailed,Use_Part,Climate_Zone_Trewartha,USDA_Hardiness_Zone,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max,Datasheet_URL,PFAF_URL
0,289,Abelmoschus esculentus,Abelmoschus,esculentus,NaN,herb,erect,annual,single stem,vegetables,...,180.0,food & beverage,vitamins,fruits,"tropical wet & dry (Aw), tropical wet (Ar), st...",5-11,5,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...
1,290,Abelmoschus manihot,Abelmoschus,manihot,NaN,shrub,erect,"annual, perennial","deciduous, multi stem","vegetables, ornamentals/turf, medicinals & aro...",...,365.0,food & beverage,vitamins,leaves,"tropical wet & dry (Aw), tropical wet (Ar)",8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...
2,291,Abelmoschus moschatus,Abelmoschus,moschatus,NaN,"herb, sub-shrub",prostrate/procumbent/semi-erect,"annual, biennial, perennial","deciduous, multi stem","ornamentals/turf, medicinals & aromatic",...,0.0,environmental,ornamental/turf,entire plant,tropical wet & dry (Aw),8-11,8,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Abe...
3,295,Acacia auriculiformis,Acacia,auriculiformis,NaN,tree,erect,perennial,single stem,forest/wood,...,240.0,material,dye/tannin,stems,"tropical wet & dry (Aw), tropical wet (Ar)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...
4,297,Acacia farnesiana,Acacia,farnesiana,NaN,tree,erect,perennial,single stem,"materials, ornamentals/turf, medicinals & arom...",...,240.0,environmental,ornamental/turf,entire plant,"tropical wet & dry (Aw), steppe or semiarid (B...",9-11,9,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,400000,Chamaecrista rotundifolia,Chamaecrista,rotundifolia,NaN,herb,prostrate/procumbent/semi-erect,"annual, perennial","deciduous, multi stem","forage/pasture, medicinals & aromatic",...,0.0,animal food (feed),vitamins,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), su...",9-11,9,11,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Cha...
2564,400001,Acacia polyacantha,Acacia,polyacantha,NaN,"shrub, tree",erect,perennial,"single stem, multi stem","forest/wood, environmental",...,0.0,fuels,fuelwood,bark,"desert or arid (Bw), steppe or semiarid (Bs)",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Aca...
2565,400002,Prosopis affinis,Prosopis,affinis,NaN,tree,erect,perennial,"deciduous, single stem, C3 photosynthesis",forest/wood,...,0.0,food & beverage,vitamins,bark,"tropical wet & dry (Aw), desert or arid (Bw), ...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Pro...
2566,400003,Vicia dasycarpa,Vicia,dasycarpa,NaN,herb,climber/scrambler/scadent,annual,multi stem,forage/pasture,...,0.0,animal food (feed),minerals,entire plant,"tropical wet & dry (Aw), tropical wet (Ar), st...",10-12,10,12,http://ecocrop.fao.org/ecocrop/srv/en/dataShee...,https://pfaf.org/user/Plant.aspx?LatinName=Vic...


In [5]:
list(ecocrop_all.columns)

['Crop_Code',
 'Scientific_Name',
 'Genus',
 'Species',
 'Variety',
 'Life_Form',
 'Habit',
 'Life_Span',
 'Physiology',
 'Category',
 'Plant_Attributes',
 'Temp_Opt_Min',
 'Temp_Opt_Max',
 'Temp_Opt_Min_F',
 'Temp_Opt_Max_F',
 'Temp_Abs_Min',
 'Temp_Abs_Max',
 'Temp_Abs_Min_F',
 'Temp_Abs_Max_F',
 'Killing_Temp_Rest',
 'Killing_Temp_Growth',
 'Killing_Temp_Rest_F',
 'Killing_Temp_Growth_F',
 'Rain_Opt_Min',
 'Rain_Opt_Max',
 'Rain_Abs_Min',
 'Rain_Abs_Max',
 'Lat_Opt_Min',
 'Lat_Opt_Max',
 'Lat_Abs_Min',
 'Lat_Abs_Max',
 'Alt_Opt_Min',
 'Alt_Opt_Max',
 'Alt_Abs_Min',
 'Alt_Abs_Max',
 'pH_Opt_Min',
 'pH_Opt_Max',
 'pH_Abs_Min',
 'pH_Abs_Max',
 'Light_Opt_Min',
 'Light_Opt_Max',
 'Light_Abs_Min',
 'Light_Abs_Max',
 'Depth_Opt',
 'Depth_Abs',
 'Texture_Ops',
 'Texture_Abs',
 'Fertility_Ops',
 'Fertility_Abs',
 'Al_Toxicity_Opt',
 'Al_Toxicity_Abs',
 'Salinity_Ops',
 'Salinity_Abs',
 'Drainage_Opt',
 'Drainage_Abs',
 'Photoperiod',
 'Abiotic_Tolererance',
 'Abiotic_Susceptibility',
 'Intr

In [6]:
crop_columns = [
    'Crop_Code',
    'Scientific_Name',
    'Genus',
    'Species',
    # 'Variety',
    # 'Life_Form',
    # 'Habit',
    # 'Life_Span',
    # 'Physiology',
    # 'Category',
    # 'Plant_Attributes',
    # 'Temp_Opt_Min',
    # 'Temp_Opt_Max',
    # 'Temp_Opt_Min_F',
    # 'Temp_Opt_Max_F',
    # 'Temp_Abs_Min',
    # 'Temp_Abs_Max',
    # 'Temp_Abs_Min_F',
    # 'Temp_Abs_Max_F',
    # 'Killing_Temp_Rest',
    # 'Killing_Temp_Growth',
    # 'Killing_Temp_Rest_F',
    # 'Killing_Temp_Growth_F',
    # 'Rain_Opt_Min',
    # 'Rain_Opt_Max',
    # 'Rain_Abs_Min',
    # 'Rain_Abs_Max',
    # 'Lat_Opt_Min',
    # 'Lat_Opt_Max',
    # 'Lat_Abs_Min',
    # 'Lat_Abs_Max',
    # 'Alt_Opt_Min',
    # 'Alt_Opt_Max',
    # 'Alt_Abs_Min',
    # 'Alt_Abs_Max',
    # 'pH_Opt_Min',
    # 'pH_Opt_Max',
    # 'pH_Abs_Min',
    # 'pH_Abs_Max',
    # 'Light_Opt_Min',
    # 'Light_Opt_Max',
    # 'Light_Abs_Min',
    # 'Light_Abs_Max',
    # 'Depth_Opt',
    # 'Depth_Abs',
    # 'Texture_Ops',
    # 'Texture_Abs',
    # 'Fertility_Ops',
    # 'Fertility_Abs',
    # 'Al_Toxicity_Opt',
    # 'Al_Toxicity_Abs',
    # 'Salinity_Ops',
    # 'Salinity_Abs',
    # 'Drainage_Opt',
    # 'Drainage_Abs',
    # 'Photoperiod',
    # 'Abiotic_Tolererance',
    # 'Abiotic_Susceptibility',
    # 'Introduction_Risks',
    # 'Production_System',
    # 'Cropping_System',
    # 'Subsystem',
    # 'Companion_Species',
    # 'Level_of_Mechanization',
    # 'Labour_Intensity',
    # 'Crop_Cycle_Min',
    # 'Crop_Cycle_Max',
    # 'Use_Main',
    # 'Use_Detailed',
    # 'Use_Part',
    # 'Climate_Zone_Trewartha',
    # 'USDA_Hardiness_Zone',
    'USDA_Hardiness_Zone_Min',
    'USDA_Hardiness_Zone_Max',
    # 'Datasheet_URL',
    # 'PFAF_URL'
]

In [7]:
crops = ecocrop_all[crop_columns]

In [8]:
crops

,Crop_Code,Scientific_Name,Genus,Species,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max
0,289,Abelmoschus esculentus,Abelmoschus,esculentus,5,11
1,290,Abelmoschus manihot,Abelmoschus,manihot,8,11
2,291,Abelmoschus moschatus,Abelmoschus,moschatus,8,11
3,295,Acacia auriculiformis,Acacia,auriculiformis,10,12
4,297,Acacia farnesiana,Acacia,farnesiana,9,11
...,...,...,...,...,...,...
2563,400000,Chamaecrista rotundifolia,Chamaecrista,rotundifolia,9,11
2564,400001,Acacia polyacantha,Acacia,polyacantha,10,12
2565,400002,Prosopis affinis,Prosopis,affinis,10,12
2566,400003,Vicia dasycarpa,Vicia,dasycarpa,10,12


In [9]:
crops.columns

Index(['Crop_Code', 'Scientific_Name', 'Genus', 'Species',
       'USDA_Hardiness_Zone_Min', 'USDA_Hardiness_Zone_Max'],
      dtype='object')

In [10]:
crops[crops['USDA_Hardiness_Zone_Min'].isna()]

,Crop_Code,Scientific_Name,Genus,Species,USDA_Hardiness_Zone_Min,USDA_Hardiness_Zone_Max


## look up hardiness zone data

In [11]:
! ls '../data/zones/usda_cold_hardiness/'

20CMIP5ModelMean_1971_2000_historical.nc
20CMIP5ModelMean_2040_2069_rcp45.nc
20CMIP5ModelMean_2040_2069_rcp85.nc


In [19]:
climate_toolbox_data_folder = '../data/zones/usda_cold_hardiness/'

hardiness_filenames = [   
    '20CMIP5ModelMean_1971_2000_historical.nc',
    '20CMIP5ModelMean_2040_2069_rcp45.nc',
    '20CMIP5ModelMean_2040_2069_rcp85.nc',
]

In [20]:
hardiness_filepaths = [f'{climate_toolbox_data_folder}{filename}' for filename in hardiness_filenames ]
hardiness_filepaths

['../data/zones/usda_cold_hardiness/20CMIP5ModelMean_1971_2000_historical.nc',
 '../data/zones/usda_cold_hardiness/20CMIP5ModelMean_2040_2069_rcp45.nc',
 '../data/zones/usda_cold_hardiness/20CMIP5ModelMean_2040_2069_rcp85.nc']

In [21]:
hardiness_scenarios = [
    'usda_plant_hardiness_1971_2000_historical',
    'usda_plant_hardiness_2040_2069_rcp45',
    'usda_plant_hardiness_2040_2069_rcp85',
]

In [22]:
hardiness_scenarios_data = {scenario:filepath for scenario, filepath in zip(hardiness_scenarios, hardiness_filepaths)} 
hardiness_scenarios_data

{'usda_plant_hardiness_1971_2000_historical': '../data/zones/usda_cold_hardiness/20CMIP5ModelMean_1971_2000_historical.nc',
 'usda_plant_hardiness_2040_2069_rcp45': '../data/zones/usda_cold_hardiness/20CMIP5ModelMean_2040_2069_rcp45.nc',
 'usda_plant_hardiness_2040_2069_rcp85': '../data/zones/usda_cold_hardiness/20CMIP5ModelMean_2040_2069_rcp85.nc'}

In [23]:
# Convert to GeoDataFrame with NAD83 CRS
gdf = gpd.GeoDataFrame(
    geo_balanced_csb_samples,
    geometry=gpd.points_from_xy(geo_balanced_csb_samples.Longitude, geo_balanced_csb_samples.Latitude),
    crs='EPSG:4269'  # NAD83
)

latitudes = gdf['Latitude']
longitudes = gdf['Longitude']

In [24]:
# Function to extract data variable names from a NetCDF file
def extract_data_variables(file_path):
    ds = xr.open_dataset(file_path)
    print(list(ds.data_vars))
    return list(ds.data_vars)

# Function to get climate values for specific locations
def get_climate_values_for_locations(latitudes, longitudes, file_path):
    ds = xr.open_dataset(file_path)

    variable_name = list(ds.data_vars)[0]
    print(f"Variable name: {variable_name}")

    # Check if the dataset has a time dimension and select the first time slice if present
    if 'time' in ds.dims:
        ds = ds.isel(time=0)

    # Ensure the variable name exists in the dataset
    if variable_name not in ds.data_vars:
        raise ValueError(f"Variable {variable_name} does not exist in the dataset.")

    # Check if longitudes in the dataset are in the range [0, 360]
    lon_min = ds.lon.min().values
    lon_max = ds.lon.max().values

    if lon_max > 180:
        # Adjust input longitudes to match the NetCDF file's 0-360 range
        longitudes = (longitudes + 360) % 360

    # Create a DataArray for the interpolation points
    points = xr.DataArray(
        np.array([latitudes, longitudes]).T,
        dims=["points", "coords"],
        coords={"points": range(len(latitudes)), "coords": ["lat", "lon"]}
    )

    # Interpolate the data
    climate_values = ds[variable_name].interp(lat=points[:, 0], lon=points[:, 1], method='linear')
    print(climate_values.values[:10])
    # Return the interpolated values for the specified variable
    return climate_values.values

In [25]:
# Time the step of adding hardiness data to the dataframe

latitudes = gdf['Latitude']
longitudes = gdf['Longitude']

start_time = time.time()
for scenario_name, file_path in hardiness_scenarios_data.items():
    # Extract column name from the scenario name
    column_name = scenario_name
    print(f"Column name: {column_name}")
    
    if column_name:
        interpolated_values = get_climate_values_for_locations(
            latitudes,
            longitudes,
            file_path,
        )
        # Ensure the length of interpolated values matches the DataFrame's index length
        if len(interpolated_values) == len(gdf):
            # print(interpolated_values[:10])
            gdf[column_name] = interpolated_values
        else:
            raise ValueError(f"Length of interpolated values ({len(interpolated_values)}) does not match the length of the DataFrame's index ({len(gdf)})")

end_time = time.time()

# Print the execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Column name: usda_plant_hardiness_1971_2000_historical
Variable name: cold_hardiness_zone
[6.         7.5        7.         6.         5.5        5.5
 7.         6.         6.11417072 6.15530776]
Column name: usda_plant_hardiness_2040_2069_rcp45
Variable name: cold_hardiness_zone
[6.5        8.         7.5        6.5        6.         6.5
 7.5        6.88226846 7.         7.        ]
Column name: usda_plant_hardiness_2040_2069_rcp85
Variable name: cold_hardiness_zone
[6.5        8.         7.56270243 6.69060233 6.5        6.5
 8.         7.         7.37645844 7.15530776]
Execution time: 0.4875788688659668 seconds


In [26]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1200 entries, 478801 to 308295
Data columns (total 29 columns):
 #   Column                                     Non-Null Count  Dtype   
---  ------                                     --------------  -----   
 0   CSBID                                      1200 non-null   object  
 1   CSBYEARS                                   1200 non-null   object  
 2   CSBACRES                                   1200 non-null   float64 
 3   CDL2016                                    1200 non-null   int64   
 4   CDL2017                                    1200 non-null   int64   
 5   CDL2018                                    1200 non-null   int64   
 6   CDL2019                                    1200 non-null   int64   
 7   CDL2020                                    1200 non-null   int64   
 8   CDL2021                                    1200 non-null   int64   
 9   CDL2022                                    1200 non-null   int64   
 10  CD

In [27]:
gdf

,CSBID,CSBYEARS,CSBACRES,CDL2016,CDL2017,CDL2018,CDL2019,CDL2020,CDL2021,CDL2022,...,Shp_Area,geometry,Longitude,Latitude,Elevation,color,Crop,usda_plant_hardiness_1971_2000_historical,usda_plant_hardiness_2040_2069_rcp45,usda_plant_hardiness_2040_2069_rcp85
478801,081623012787392,1623,3.791540,4,24,61,61,24,24,61,...,15343.877410,POINT (-102.96208 38.17692),-102.962080,38.176922,1231,#bfbf7a,Fallow/Idle Cropland,6.0,6.500000,6.500000
107446,351623001627247,1623,4.325446,225,1,225,152,152,152,36,...,17504.529797,POINT (-104.60067 33.08984),-104.600667,33.089835,1138,#ffa8e3,Alfalfa,7.5,8.000000,8.000000
121162,351623002777545,1623,3.222010,61,176,1,176,36,36,1,...,13039.063523,POINT (-108.30975 36.72811),-108.309751,36.728108,1676,#ffd400,Corn,7.0,7.500000,7.562702
88813,351623000046581,1623,10.204036,36,36,36,36,152,37,37,...,41294.433943,POINT (-107.11218 36.89108),-107.112180,36.891083,2252,#a5f58d,Other Hay/Non Alfalfa,6.0,6.500000,6.690602
516266,081623013377366,1623,2.918481,24,29,29,24,1,61,61,...,11810.721459,POINT (-103.41289 40.72386),-103.412893,40.723858,1330,#bfbf7a,Fallow/Idle Cropland,5.5,6.000000,6.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119071,351623002568175,1623,2.957066,176,176,176,61,24,24,24,...,11966.871324,POINT (-106.08262 34.75611),-106.082615,34.756107,1877,#e9ffbe,Grass/Pasture,6.5,7.000000,7.500000
540278,081623014401456,1623,9.374885,61,24,61,1,1,1,1,...,37938.965442,POINT (-104.17285 40.16143),-104.172849,40.161427,1417,#a5f58d,Other Hay/Non Alfalfa,5.5,6.000000,6.102624
591949,041623013556134,1623,3.770585,72,72,72,190,190,190,190,...,15259.076277,POINT (-114.50854 33.02805),-114.508535,33.028054,101,#80b3b3,Woody Wetlands,10.0,10.000000,10.500000
217842,081623005639708,1623,4.307316,61,24,1,61,24,29,61,...,17431.160158,POINT (-103.09239 39.90366),-103.092389,39.903658,1396,#a87000,Winter Wheat,5.5,6.008018,6.500000


In [28]:
fields = gdf.copy()

In [29]:
for column in hardiness_scenarios:
    print(f'{column:>60} {gdf[column].min():>8.2f} (Min),   {gdf[column].max():>8.1f} (Max)')

                   usda_plant_hardiness_1971_2000_historical     4.50 (Min),       10.0 (Max)
                        usda_plant_hardiness_2040_2069_rcp45     5.00 (Min),       10.0 (Max)
                        usda_plant_hardiness_2040_2069_rcp85     5.06 (Min),       10.5 (Max)


In [30]:
crops.columns

Index(['Crop_Code', 'Scientific_Name', 'Genus', 'Species',
       'USDA_Hardiness_Zone_Min', 'USDA_Hardiness_Zone_Max'],
      dtype='object')

In [31]:
crop_hardiness_columns = ['USDA_Hardiness_Zone_Min', 'USDA_Hardiness_Zone_Max']

for column in crop_hardiness_columns:
    print(f'{column:>60} {crops[column].min():>8.2f} (Min),   {crops[column].max():>8.1f} (Max)')

                                     USDA_Hardiness_Zone_Min     1.00 (Min),       11.0 (Max)
                                     USDA_Hardiness_Zone_Max     5.00 (Min),       12.0 (Max)


In [32]:
scenario_names = [
    'historical', 
    'mid_century_medium_CO2', 
    'mid_century_high_CO2'
]

scenario_columns = [
    'usda_plant_hardiness_1971_2000_historical',
    'usda_plant_hardiness_2040_2069_rcp45',
    'usda_plant_hardiness_2040_2069_rcp85',
]

In [33]:
crops.columns

Index(['Crop_Code', 'Scientific_Name', 'Genus', 'Species',
       'USDA_Hardiness_Zone_Min', 'USDA_Hardiness_Zone_Max'],
      dtype='object')

In [34]:
# Example initialization of data (replace with actual data)
num_fields = len(fields)
num_crops = len(crops)
num_scenarios = len(scenario_names)
hardiness_scores = np.zeros((num_fields, num_crops, num_scenarios))

# Extract crop-specific data
crop_hardiness_min = crops['USDA_Hardiness_Zone_Min'].values[np.newaxis, :].to_numpy()  # Shape: (1, num_crops)
crop_hardiness_max = crops['USDA_Hardiness_Zone_Max'].values[np.newaxis, :].to_numpy()  # Shape: (1, num_crops)

# Iterate over scenarios
for scenario_idx, (scenario_name, scenario_column) in enumerate(zip(scenario_names, scenario_columns)):
    start_time = time.time()
    
    # Extract the relevant average daily precipitation data for the scenario
    field_zone = fields[scenario_column].values[:, np.newaxis]  # Shape: (num_fields, 1)
    
    # Apply vectorized comparisons
    within_range = (field_zone >= crop_hardiness_min) & (field_zone <= crop_hardiness_max)
    # missing_data = (crop_hardiness_min == None) | (crop_hardiness_max == None) | (field_zone == None)
    missing_data = np.isnan(crop_hardiness_min) | np.isnan(crop_hardiness_max) | np.isnan(field_zone)
    
    # Initialize scores
    scores = np.zeros((num_fields, num_crops), dtype=int)
    
    # Assign scores
    scores[within_range] = 1
    scores[missing_data] = 0
    scores[~within_range & ~missing_data] = -1
    
    # Store the scores in the matrix
    hardiness_scores[:, :, scenario_idx] = scores

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time:.6f} seconds for scenario {scenario_name}")


Execution time: 0.060758 seconds for scenario historical
Execution time: 0.029215 seconds for scenario mid_century_medium_CO2
Execution time: 0.022914 seconds for scenario mid_century_high_CO2


In [35]:
np.unique(hardiness_scores, return_counts=True)

(array([-1.,  1.]), array([6835776, 2409024]))

In [36]:
scenarios = ['historical', 'mid_century_medium_CO2', 'mid_century_high_CO2']

# Convert the results matrix to an xarray DataArray
results_da = xr.DataArray(
    hardiness_scores,
    dims=['fields', 'crops', 'scenarios'],
    coords={
        'fields': fields['CSBID'],
        'crops': crops['Scientific_Name'],
        'scenarios': scenarios,
    },
    name='crop_suitability_usda_hardiness'
)

In [37]:
# Save the DataArray to a NetCDF file
results_da.to_netcdf('../data/scores/score_matrix_hardiness.nc')

## save field usda hardiness data

In [38]:
fields.to_crs(crs='EPSG:4269', inplace=True)

In [39]:
fields.to_parquet('../data/fields/csb_sample_with_hardiness_data.parquet')

In [40]:
fields['CSBID'].info()

<class 'pandas.core.series.Series'>
Index: 1200 entries, 478801 to 308295
Series name: CSBID
Non-Null Count  Dtype 
--------------  ----- 
1200 non-null   object
dtypes: object(1)
memory usage: 18.8+ KB


In [41]:
hardiness_scores.shape

(1200, 2568, 3)

In [42]:
## analyze temperature recommendations

In [43]:
def get_crops_with_score_1(scores_matrix, crop_data):
    suitable_crops = {}
    for field_idx in range(scores_matrix.shape[0]):
        suitable_crops[field_idx] = [crop_data['Scientific_Name'][crop_idx] for crop_idx in range(scores_matrix.shape[1]) if scores_matrix[field_idx, crop_idx] == 1]
    return suitable_crops

In [44]:
hardiness_scores

array([[[ 1.,  1.,  1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [ 1.,  1.,  1.]],

       [[ 1.,  1.,  1.],
        [-1.,  1.,  1.],
        [-1.,  1.,  1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [ 1.,  1.,  1.]],

       [[ 1.,  1.,  1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [ 1.,  1.,  1.]],

       ...,

       [[ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.],
        [ 1.,  1.,  1.],
        [-1., -1., -1.]],

       [[ 1.,  1.,  1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1.,  1.,  1.]],

       [[ 1.,  1.,  1.],
        [-1., -1., -1.],
        [-1., -1., -1.],
        ...,
        [-1., -1., -1.],
        [-1., -1., -1.],
        [-1.,  1.,  1.]]

In [45]:
# fields = gdf.copy()

In [46]:
# Initialize the DataFrame to store the results using CSBID from fields
crops_with_score_1_df = pd.DataFrame({'CSBID': range(hardiness_scores.shape[0])})

# Process each scenario and add the results to the DataFrame
for scenario_idx, scenario in enumerate(scenario_names):
    # Extract crops with a score of 1 for this scenario
    crops_with_score_1 = get_crops_with_score_1(hardiness_scores[:, :, scenario_idx], crops)
    
    # Convert the results to a DataFrame for easier visualization
    scenario_column = f'Suitable_Crops_{scenario}'
    number_column = f'Number_of_Suitable_Crops_{scenario}'
    temp_df = pd.DataFrame(list(crops_with_score_1.items()), columns=['CSBID', scenario_column])
    temp_df[number_column] = temp_df[scenario_column].apply(len)

    # Merge with the main DataFrame
    crops_with_score_1_df = crops_with_score_1_df.merge(temp_df, on='CSBID', how='outer')

crops_with_score_1_df['CSBID'] = list(fields['CSBID'])

# Output the resulting DataFrame
# print(crops_with_score_1_df)

In [47]:
crops_with_score_1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   CSBID                                            1200 non-null   object
 1   Suitable_Crops_historical                        1200 non-null   object
 2   Number_of_Suitable_Crops_historical              1200 non-null   int64 
 3   Suitable_Crops_mid_century_medium_CO2            1200 non-null   object
 4   Number_of_Suitable_Crops_mid_century_medium_CO2  1200 non-null   int64 
 5   Suitable_Crops_mid_century_high_CO2              1200 non-null   object
 6   Number_of_Suitable_Crops_mid_century_high_CO2    1200 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 65.8+ KB


In [48]:
data_for_plot = crops_with_score_1_df[['CSBID', 'Number_of_Suitable_Crops_historical', 'Number_of_Suitable_Crops_mid_century_medium_CO2', 'Number_of_Suitable_Crops_mid_century_high_CO2',]]
data_for_plot
# data_for_plot.info()

,CSBID,Number_of_Suitable_Crops_historical,Number_of_Suitable_Crops_mid_century_medium_CO2,Number_of_Suitable_Crops_mid_century_high_CO2
0,081623012787392,639,631,631
1,351623001627247,661,816,816
2,351623002777545,742,661,661
3,351623000046581,639,631,631
4,081623013377366,531,639,631
...,...,...,...,...
1195,351623002568175,631,661,661
1196,081623014401456,531,639,631
1197,041623013556134,2035,2035,1868
1198,081623005639708,531,631,631


In [49]:
crop_suitability_columns = [
    'Number_of_Suitable_Crops_historical', 
    'Number_of_Suitable_Crops_mid_century_medium_CO2', 
    'Number_of_Suitable_Crops_mid_century_high_CO2', 
]

for column in crop_suitability_columns:
    print(f'{column:>50}  {crops_with_score_1_df[column].min():>8.2f} (Min),  {crops_with_score_1_df[column].max():>8.1f} (Max),  {crops_with_score_1_df[column].mean():>8.1f} (Avg)')

               Number_of_Suitable_Crops_historical    394.00 (Min),    2035.0 (Max),     622.5 (Avg)
   Number_of_Suitable_Crops_mid_century_medium_CO2    531.00 (Min),    2035.0 (Max),     676.8 (Avg)
     Number_of_Suitable_Crops_mid_century_high_CO2    531.00 (Min),    2035.0 (Max),     708.1 (Avg)


In [51]:
# Set bin size
bin_size = 20  # You can adjust the bin size as needed
color='cornflowerblue'
opacity=0.6

# Create Altair charts for each scenario
chart_h = alt.Chart(data_for_plot).mark_bar(color=color, opacity=opacity).encode(
    x=alt.X('Number_of_Suitable_Crops_historical:Q', bin=alt.Bin(step=bin_size), title='Number of Suitable Crops (Historical 1970-2000)', axis=alt.Axis(offset=10)),
    y=alt.Y('count()', title='Number of Fields', axis=alt.Axis(offset=10)),
).properties(
    title='Historical Scenario (1970-2000)',
    width=600,
    height=200
)

chart_45 = alt.Chart(data_for_plot).mark_bar(color=color, opacity=opacity).encode(
    x=alt.X('Number_of_Suitable_Crops_mid_century_medium_CO2:Q', bin=alt.Bin(step=bin_size), title='Number of Suitable Crops (RCP 4.5, 2036-2065)', axis=alt.Axis(offset=10)),
    y=alt.Y('count()', title='Number of Fields', axis=alt.Axis(offset=10)),
).properties(
    title='RCP 4.5 Scenario (2036-2065)',
    width=600,
    height=200
)

chart_85 = alt.Chart(data_for_plot).mark_bar(color=color, opacity=opacity).encode(
    x=alt.X('Number_of_Suitable_Crops_mid_century_high_CO2:Q', bin=alt.Bin(step=bin_size), title='Number of Suitable Crops (RCP 8.5, 2036-2065)', axis=alt.Axis(offset=10)),
    y=alt.Y('count()', title='Number of Fields', axis=alt.Axis(offset=10)),
).properties(
    title='RCP 8.5 Scenario (2036-2065)',
    width=600,
    height=200
)

# Combine the charts with shared axis domains
combined_chart = alt.vconcat(
    chart_h, 
    chart_45, 
    chart_85
).resolve_scale(
    x='shared',
    y='shared'
)

combined_chart

alt.VConcatChart(...)